In [177]:
import pandas as pd
import pickle
from feature_engine.outliers import Winsorizer
from sklearn.preprocessing import StandardScaler , OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import confusion_matrix ,f1_score, accuracy_score, classification_report

In [178]:
df = pd.read_csv('fakerdata.csv')
pd.set_option("display.max_columns",None)

In [179]:
df

,eid,age,gender,maritalstatus,department,jobrole,education,environmentsatisfaction,jobinvolvement,joblevel,jobsatisfaction,Annualincome,relationshipsatisfaction,workinghrsperday,totalworkingyearsexperience,TrainingTime,WorkLifeBalance,BehaviourialCompetence,OntimeDelivery,TicketSolvingManagements,Projectevlaution,Workingfromhomeoroffice,Psychosocialindicators,overtime,attendance,effectedwithcorona,PercentSalaryHike,Netconnectivity,PerformanceRating
0,LZJ048,49,f,single,Sales,Sales Executive,4,1,2,4,4,20,3,8,8,5,4,3,1,1,14,office,5,no,present,no,6,yes,1
1,pEQ793,40,f,married,Sales,Sales Executive,3,4,1,5,4,11,2,8,24,2,4,5,5,2,12,home,4,yes,absent,no,8,yes,4
2,jGb879,33,f,single,Sales,Sales Executive,5,4,2,5,1,9,2,8,20,2,3,1,5,1,13,office,1,no,present,yes,5,yes,1
3,HXZ392,28,f,married,Human Resources,Manager,4,2,1,2,1,7,4,8,4,4,2,3,2,1,14,office,0,no,absent,yes,12,no,4
4,wTb124,45,m,married,Sales,Sales Executive,2,3,4,5,2,24,1,8,16,1,3,2,4,1,11,home,0,yes,present,no,9,no,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,lqv086,30,m,single,Research & Development,Research Scientist,5,3,4,3,4,15,2,8,8,2,4,2,5,2,14,office,4,no,absent,no,11,no,1
4996,vcu089,37,m,married,Sales,Sales Executive,2,1,2,2,1,5,2,8,25,5,3,5,1,5,8,office,2,no,present,no,2,yes,2
4997,lfe187,25,f,single,Research & Development,Senior Manager R&D,2,3,3,5,3,6,3,8,18,2,2,5,5,1,5,home,5,no,present,yes,13,no,2
4998,rjL459,41,m,single,Research & Development,Manufacturing Director,2,2,2,3,1,17,3,8,23,2,3,3,2,5,7,office,5,yes,absent,no,15,no,3


In [180]:
col = ["eid", "age", "education", "environmentsatisfaction", "jobinvolvement", "joblevel", "jobsatisfaction",
        "Annualincome", "relationshipsatisfaction", "totalworkingyearsexperience", "TrainingTime",
        "WorkLifeBalance", "BehaviourialCompetence", "OntimeDelivery", "TicketSolvingManagements",
        "Projectevlaution", "Psychosocialindicators", "PercentSalaryHike", "Netconnectivity",
        "gender", "maritalstatus", "department","jobrole", "Workingfromhomeoroffice", "overtime",
        "attendance", "effectedwithcorona",  "workinghrsperday", "PerformanceRating"]
df = df.reindex(columns = col)

In [181]:
df

,eid,age,education,environmentsatisfaction,jobinvolvement,joblevel,jobsatisfaction,Annualincome,relationshipsatisfaction,totalworkingyearsexperience,TrainingTime,WorkLifeBalance,BehaviourialCompetence,OntimeDelivery,TicketSolvingManagements,Projectevlaution,Psychosocialindicators,PercentSalaryHike,Netconnectivity,gender,maritalstatus,department,jobrole,Workingfromhomeoroffice,overtime,attendance,effectedwithcorona,workinghrsperday,PerformanceRating
0,LZJ048,49,4,1,2,4,4,20,3,8,5,4,3,1,1,14,5,6,yes,f,single,Sales,Sales Executive,office,no,present,no,8,1
1,pEQ793,40,3,4,1,5,4,11,2,24,2,4,5,5,2,12,4,8,yes,f,married,Sales,Sales Executive,home,yes,absent,no,8,4
2,jGb879,33,5,4,2,5,1,9,2,20,2,3,1,5,1,13,1,5,yes,f,single,Sales,Sales Executive,office,no,present,yes,8,1
3,HXZ392,28,4,2,1,2,1,7,4,4,4,2,3,2,1,14,0,12,no,f,married,Human Resources,Manager,office,no,absent,yes,8,4
4,wTb124,45,2,3,4,5,2,24,1,16,1,3,2,4,1,11,0,9,no,m,married,Sales,Sales Executive,home,yes,present,no,8,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,lqv086,30,5,3,4,3,4,15,2,8,2,4,2,5,2,14,4,11,no,m,single,Research & Development,Research Scientist,office,no,absent,no,8,1
4996,vcu089,37,2,1,2,2,1,5,2,25,5,3,5,1,5,8,2,2,yes,m,married,Sales,Sales Executive,office,no,present,no,8,2
4997,lfe187,25,2,3,3,5,3,6,3,18,2,2,5,5,1,5,5,13,no,f,single,Research & Development,Senior Manager R&D,home,no,present,yes,8,2
4998,rjL459,41,2,2,2,3,1,17,3,23,2,3,3,2,5,7,5,15,no,m,single,Research & Development,Manufacturing Director,office,yes,absent,no,8,3


In [182]:
df.shape

(5000, 29)

In [183]:
df = df.drop(['eid','workinghrsperday'], axis = 1)
df.columns

Index(['age', 'education', 'environmentsatisfaction', 'jobinvolvement',
       'joblevel', 'jobsatisfaction', 'Annualincome',
       'relationshipsatisfaction', 'totalworkingyearsexperience',
       'TrainingTime', 'WorkLifeBalance', 'BehaviourialCompetence',
       'OntimeDelivery', 'TicketSolvingManagements', 'Projectevlaution',
       'Psychosocialindicators', 'PercentSalaryHike', 'Netconnectivity',
       'gender', 'maritalstatus', 'department', 'jobrole',
       'Workingfromhomeoroffice', 'overtime', 'attendance',
       'effectedwithcorona', 'PerformanceRating'],
      dtype='object')

In [184]:
features = df.drop(columns = ["PerformanceRating"], axis = 1).columns
pickle.dump(features, open("features.pkl", "wb"))

In [185]:
features

Index(['age', 'education', 'environmentsatisfaction', 'jobinvolvement',
       'joblevel', 'jobsatisfaction', 'Annualincome',
       'relationshipsatisfaction', 'totalworkingyearsexperience',
       'TrainingTime', 'WorkLifeBalance', 'BehaviourialCompetence',
       'OntimeDelivery', 'TicketSolvingManagements', 'Projectevlaution',
       'Psychosocialindicators', 'PercentSalaryHike', 'Netconnectivity',
       'gender', 'maritalstatus', 'department', 'jobrole',
       'Workingfromhomeoroffice', 'overtime', 'attendance',
       'effectedwithcorona'],
      dtype='object')

In [186]:
cols_encode = ["Netconnectivity", "gender", "maritalstatus", "department", "jobrole",
                "Workingfromhomeoroffice", "overtime", "attendance", "effectedwithcorona"]

In [187]:
enc_label = OneHotEncoder(sparse= False).fit(df.loc[:, cols_encode])
pickle.dump(enc_label, open("encoder.pkl", "wb"))

In [188]:
column_names = enc_label.get_feature_names_out(cols_encode)
column_names

array(['Netconnectivity_no', 'Netconnectivity_yes', 'gender_f',
       'gender_m', 'maritalstatus_married', 'maritalstatus_single',
       'department_Data Science', 'department_Development',
       'department_Finance', 'department_Human Resources',
       'department_Research & Development', 'department_Sales',
       'jobrole_Business Analyst', 'jobrole_Data Scientist',
       'jobrole_Delivery Manager', 'jobrole_Developer',
       'jobrole_Finance Manager', 'jobrole_Healthcare Representative',
       'jobrole_Human Resources', 'jobrole_Laboratory Technician',
       'jobrole_Manager', 'jobrole_Manager R&D',
       'jobrole_Manufacturing Director', 'jobrole_Research Director',
       'jobrole_Research Scientist', 'jobrole_Sales Executive',
       'jobrole_Sales Representative', 'jobrole_Senior Developer',
       'jobrole_Senior Manager R&D', 'jobrole_Technical Architect',
       'jobrole_Technical Lead', 'Workingfromhomeoroffice_home',
       'Workingfromhomeoroffice_office', 'overt

In [189]:
enc_df = pd.DataFrame(enc_label.transform(df.loc[:, cols_encode]), columns= column_names)

In [190]:
enc_df

,Netconnectivity_no,Netconnectivity_yes,gender_f,gender_m,maritalstatus_married,maritalstatus_single,department_Data Science,department_Development,department_Finance,department_Human Resources,department_Research & Development,department_Sales,jobrole_Business Analyst,jobrole_Data Scientist,jobrole_Delivery Manager,jobrole_Developer,jobrole_Finance Manager,jobrole_Healthcare Representative,jobrole_Human Resources,jobrole_Laboratory Technician,jobrole_Manager,jobrole_Manager R&D,jobrole_Manufacturing Director,jobrole_Research Director,jobrole_Research Scientist,jobrole_Sales Executive,jobrole_Sales Representative,jobrole_Senior Developer,jobrole_Senior Manager R&D,jobrole_Technical Architect,jobrole_Technical Lead,Workingfromhomeoroffice_home,Workingfromhomeoroffice_office,overtime_no,overtime_yes,attendance_absent,attendance_present,effectedwithcorona_no,effectedwithcorona_yes
0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
1,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
3,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
4,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
4996,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
4997,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4998,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0


In [191]:
df = df.drop(columns = cols_encode, axis = 1)
df = pd.concat([df, enc_df], axis = 1)

In [192]:
df

,age,education,environmentsatisfaction,jobinvolvement,joblevel,jobsatisfaction,Annualincome,relationshipsatisfaction,totalworkingyearsexperience,TrainingTime,WorkLifeBalance,BehaviourialCompetence,OntimeDelivery,TicketSolvingManagements,Projectevlaution,Psychosocialindicators,PercentSalaryHike,PerformanceRating,Netconnectivity_no,Netconnectivity_yes,gender_f,gender_m,maritalstatus_married,maritalstatus_single,department_Data Science,department_Development,department_Finance,department_Human Resources,department_Research & Development,department_Sales,jobrole_Business Analyst,jobrole_Data Scientist,jobrole_Delivery Manager,jobrole_Developer,jobrole_Finance Manager,jobrole_Healthcare Representative,jobrole_Human Resources,jobrole_Laboratory Technician,jobrole_Manager,jobrole_Manager R&D,jobrole_Manufacturing Director,jobrole_Research Director,jobrole_Research Scientist,jobrole_Sales Executive,jobrole_Sales Representative,jobrole_Senior Developer,jobrole_Senior Manager R&D,jobrole_Technical Architect,jobrole_Technical Lead,Workingfromhomeoroffice_home,Workingfromhomeoroffice_office,overtime_no,overtime_yes,attendance_absent,attendance_present,effectedwithcorona_no,effectedwithcorona_yes
0,49,4,1,2,4,4,20,3,8,5,4,3,1,1,14,5,6,1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
1,40,3,4,1,5,4,11,2,24,2,4,5,5,2,12,4,8,4,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,33,5,4,2,5,1,9,2,20,2,3,1,5,1,13,1,5,1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
3,28,4,2,1,2,1,7,4,4,4,2,3,2,1,14,0,12,4,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
4,45,2,3,4,5,2,24,1,16,1,3,2,4,1,11,0,9,3,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,30,5,3,4,3,4,15,2,8,2,4,2,5,2,14,4,11,1,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
4996,37,2,1,2,2,1,5,2,25,5,3,5,1,5,8,2,2,2,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
4997,25,2,3,3,5,3,6,3,18,2,2,5,5,1,5,5,13,2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4998,41,2,2,2,3,1,17,3,23,2,3,3,2,5,7,5,15,3,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0


In [193]:
y = df.loc[:, "PerformanceRating"]
X = df.drop(columns = 'PerformanceRating', axis=1)

In [194]:
print(len(X.columns))

56


In [195]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=10)

In [196]:
col_out = ['age', 'education', 'environmentsatisfaction', 'jobinvolvement', 'joblevel', 'jobsatisfaction',
            'Annualincome', 'relationshipsatisfaction', 'totalworkingyearsexperience', 'TrainingTime',
            'WorkLifeBalance', 'BehaviourialCompetence', 'OntimeDelivery', 'TicketSolvingManagements',
            'Projectevlaution', 'Psychosocialindicators', 'PercentSalaryHike']

In [197]:
out_tre = Winsorizer(capping_method="iqr", tail= "both", fold = 1.5 , variables= col_out)
X_train = out_tre.fit_transform(X_train)
X_test = out_tre.transform(X_test)

In [198]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [199]:
modelknn = KNeighborsClassifier(n_neighbors=1,  weights='distance', algorithm='auto',
                             leaf_size=10, p=2, metric='minkowski', metric_params=None,
                             n_jobs = None)

In [200]:
model = BaggingClassifier(base_estimator = modelknn,
                          n_estimators=10, max_samples=1.0, max_features=1.0,
                          bootstrap=False, bootstrap_features=True, oob_score=False,
                          warm_start=True, n_jobs=None, random_state=None, verbose=0)

In [201]:
model.fit(X_train, y_train)
y_tra = model.predict(X_train)
y_predict = model.predict(X_test)

In [202]:
print(accuracy_score(y_train,y_tra))
print(accuracy_score(y_test,y_predict))
print('F1 Score: %.2f' % f1_score(y_test, y_predict, average='macro'))
print(classification_report(y_test,y_predict))
confusion_matrix(y_test,y_predict)

1.0
0.5253333333333333
F1 Score: 0.53
              precision    recall  f1-score   support

           1       0.48      0.57      0.52       381
           2       0.54      0.53      0.53       375
           3       0.51      0.48      0.50       369
           4       0.58      0.51      0.55       375

    accuracy                           0.53      1500
   macro avg       0.53      0.52      0.53      1500
weighted avg       0.53      0.53      0.53      1500



array([[217,  61,  66,  37],
       [ 78, 200,  48,  49],
       [ 83,  55, 178,  53],
       [ 70,  57,  55, 193]], dtype=int64)

In [203]:
pickle.dump(model, open("emp.pkl", "wb"))